In [1]:
import os
import yaml
import pandas as pd
from collections import defaultdict
import subprocess
from tqdm import tqdm
from bioblocks.io import read_model, write_model

# ===================== 读取 YAML =====================
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

common_params = config.get("common", {})
af3_params = config.get("af3_results_ppi", {})

CHAIN_TARGET = common_params.get("CHAIN_TARGET", "A")
CHAIN_QUERY = common_params.get("CHAIN_QUERY", "H,L")
NO_PYMOL = common_params.get("NO_PYMOL", True)

CIF_FOLDER = af3_params["CIF_FOLDER"]
PDB_FOLDER = af3_params["PDB_FOLDER"]
RMSD_FILE = af3_params["RMSD_FILE"]
PPI_FOLDER = af3_params["PPI_FOLDER"]
PPI_SUMMARY = af3_params["PPI_SUMMARY"]
MAPPING_FILE = af3_params["MAPPING_FILE"]
OUTPUT_MAPPING = af3_params["OUTPUT_MAPPING"]

os.makedirs(PDB_FOLDER, exist_ok=True)
os.makedirs(PPI_FOLDER, exist_ok=True)
os.makedirs(os.path.dirname(PPI_SUMMARY), exist_ok=True)
os.makedirs(os.path.dirname(OUTPUT_MAPPING), exist_ok=True)

In [3]:
# ===================== 1️⃣ CIF -> PDB (RMSD <= 60) =====================
rmsd_df = pd.read_csv(RMSD_FILE)
valid_files = set(rmsd_df[rmsd_df["RMSD"] <= 60]["file"].tolist())

for cif_file in os.listdir(CIF_FOLDER):
    if cif_file.endswith(".cif") and cif_file in valid_files:
        cif_path = os.path.join(CIF_FOLDER, cif_file)
        pdb_path = os.path.join(PDB_FOLDER, cif_file.replace(".cif", ".pdb"))
        model = read_model(cif_path)
        write_model(model, pdb_path)
        print(f"✅ Converted: {cif_file}")

✅ Converted: MJ00D_94.cif
✅ Converted: MJ00D_172.cif
✅ Converted: MJ00D_53.cif
✅ Converted: MJ00D_317.cif
✅ Converted: MJ00D_28.cif
✅ Converted: MJ00D_226.cif
✅ Converted: MJ00D_152.cif
✅ Converted: MJ00D_111.cif
✅ Converted: MJ00D_200.cif
✅ Converted: MJ00D_137.cif
✅ Converted: MJ00D_154.cif
✅ Converted: MJ00D_131.cif
✅ Converted: MJ00D_114.cif
✅ Converted: MJ00D_133.cif
✅ Converted: MJ00D_100.cif
✅ Converted: MJ00D_252.cif
✅ Converted: MJ00D_55.cif
✅ Converted: MJ00D_316.cif
✅ Converted: MJ00D_285.cif
✅ Converted: MJ00D_279.cif
✅ Converted: MJ00D_173.cif
✅ Converted: MJ00D_389.cif
✅ Converted: MJ00D_202.cif
✅ Converted: MJ00D_147.cif
✅ Converted: MJ00D_277.cif
✅ Converted: MJ00D_177.cif
✅ Converted: MJ00D_167.cif
✅ Converted: MJ00D_361.cif
✅ Converted: MJ00D_264.cif
✅ Converted: MJ00D_98.cif
✅ Converted: MJ00D_122.cif
✅ Converted: MJ00D_371.cif
✅ Converted: MJ00D_124.cif
✅ Converted: MJ00D_295.cif
✅ Converted: MJ00D_240.cif
✅ Converted: MJ00D_350.cif
✅ Converted: MJ00D_359.cif
✅ Conv

In [4]:
# ===================== 2️⃣ 计算 PPI =====================
pdb_files = [f for f in os.listdir(PDB_FOLDER) if f.endswith(".pdb")]
print(f"将处理 {len(pdb_files)} 个 PDB 文件...\n")

for pdb_file in tqdm(pdb_files):
    pdb_path = os.path.join(PDB_FOLDER, pdb_file)
    cmd = [
        "ppi.analyse",
        pdb_path,
        PPI_FOLDER,
        "--no-pymol" if NO_PYMOL else "",
        "--chain-target", CHAIN_TARGET,
        "--chain-query", CHAIN_QUERY
    ]
    # 过滤掉空参数
    cmd = [c for c in cmd if c]
    subprocess.run(cmd, check=True)
    print(f"✅ 已完成: {pdb_file}")


将处理 78 个 PDB 文件...



  0%|          | 0/78 [00:00<?, ?it/s]/home/yuyang/.conda/envs/ppi/lib/python3.10/site-packages/MDAnalysis/topology/tables.py:52: DeprecationWarning: Deprecated in version 2.8.0
MDAnalysis.topology.tables has been moved to MDAnalysis.guesser.tables. This import point will be removed in MDAnalysis version 3.0.0
  warnings.warn(wmsg, category=DeprecationWarning)
Processing PDB files:   0%|          | 0/1 [00:00<?, ?it/s]


Found 1 PDB file(s)

Will process 1 file(s)
Running 1 PDB files with 1 processes
Processing /home/yuyang/lb_yaml/data/IL23/cif_pdb/MJ00D_374.pdb
Available chains in /home/yuyang/lb_yaml/data/IL23/cif_pdb/MJ00D_374.pdb: ['A', 'L', 'H']
Analysing ['A'] vs ['H', 'L']
[('segid A', 'segid H or segid L')]


[23:27:40] Explicit valence for atom # 439 N, 4, is greater than permitted
/home/yuyang/.conda/envs/ppi/lib/python3.10/site-packages/MDAnalysis/converters/RDKit.py:527: UserWarning: Could not sanitize molecule: failed during step rdkit.Chem.rdmolops.SanitizeFlags.SANITIZE_PROPERTIES
  warnings.warn(
[23:27:41] Explicit valence for atom # 591 N, 4, is greater than permitted


Successfully processed segid A vs segid H or segid L

Outputs saved to: /home/yuyang/lb_yaml/data/IL23/ppi_csv_af3


...


Found 1 PDB file(s)

Will process 1 file(s)
Running 1 PDB files with 1 processes
Processing /home/yuyang/lb_yaml/data/IL23/cif_pdb/MJ00D_240.pdb
Available chains in /home/yuyang/lb_yaml/data/IL23/cif_pdb/MJ00D_240.pdb: ['A', 'H', 'L']
Analysing ['A'] vs ['H', 'L']
[('segid A', 'segid H or segid L')]


[23:34:49] Explicit valence for atom # 439 N, 4, is greater than permitted
/home/yuyang/.conda/envs/ppi/lib/python3.10/site-packages/MDAnalysis/converters/RDKit.py:527: UserWarning: Could not sanitize molecule: failed during step rdkit.Chem.rdmolops.SanitizeFlags.SANITIZE_PROPERTIES
  warnings.warn(
[23:34:50] Explicit valence for atom # 911 N, 4, is greater than permitted


Successfully processed segid A vs segid H or segid L

Outputs saved to: /home/yuyang/lb_yaml/data/IL23/ppi_csv_af3


100%|██████████| 78/78 [07:14<00:00,  5.58s/it]

✅ 已完成: MJ00D_240.pdb


In [5]:
# ===================== 3️⃣ 汇总 AF3 的结合位点 =====================
model_to_residues = defaultdict(set)

for csv_file in sorted(os.listdir(PPI_FOLDER)):
    if not csv_file.endswith("_interactions.csv"):
        continue
    csv_path = os.path.join(PPI_FOLDER, csv_file)
    try:
        df = pd.read_csv(csv_path)
    except Exception as e:
        print(f"⚠️ 无法读取 {csv_file}: {e}")
        continue
    df_antigen = df[df["chainA"] == CHAIN_TARGET]
    if df_antigen.empty:
        continue
    model_name = str(df_antigen["model_name"].iloc[0]).strip() if "model_name" in df_antigen.columns else os.path.basename(csv_file).split("_")[0]
    residues = df_antigen["resiA"].dropna().astype(str).str.replace(r"\D", "", regex=True).astype(int).tolist()
    model_to_residues[model_name].update(residues)

summary_data = [{"model_name": k, "antigen_binding_sites_AF3": ",".join(map(str, sorted(v)))} for k, v in sorted(model_to_residues.items())]
df_summary = pd.DataFrame(summary_data)
df_summary.to_csv(PPI_SUMMARY, index=False)
print(f"✅ 输出文件: {PPI_SUMMARY}")

✅ 输出文件: /home/yuyang/lb_yaml/data/IL23/ppi_summary_af3.csv


In [6]:
# ===================== 4️⃣ AF3 -> GD 映射 =====================
af3_df = pd.read_csv(PPI_SUMMARY)
mapping_df = pd.read_csv(MAPPING_FILE)

# 去掉列名后缀
mapping_df.columns = [col.replace("_for_msa.aln", "") if col != "pdb_index" else col for col in mapping_df.columns]

def map_to_groundtruth(model_name, antigen_sites_str):
    if model_name not in mapping_df.columns:
        return ""
    try:
        antigen_sites = [int(x) for x in str(antigen_sites_str).split(",") if x.strip()]
    except ValueError:
        return ""
    mapped = []
    for res in antigen_sites:
        row = mapping_df.loc[mapping_df[model_name] == res, "pdb_index"]
        if not row.empty:
            mapped.extend(row.tolist())
    mapped = sorted(set(mapped))
    return ",".join(map(str, mapped))

af3_df["antigen_binding_sites_groundtruth"] = af3_df.apply(
    lambda row: map_to_groundtruth(row["model_name"], row["antigen_binding_sites_AF3"]),
    axis=1
)
af3_df.to_csv(OUTPUT_MAPPING, index=False)
print(f"✅ 映射完成，输出文件：{OUTPUT_MAPPING}")

✅ 映射完成，输出文件：/home/yuyang/lb_yaml/data/IL23/ppi_summary_mapping.csv
